Завантаження Даних

In [1]:
import os
import pandas as pd

# Вкажіть шлях до каталогу з даними
data_dir = './data'

# Створіть порожній DataFrame для зберігання всіх даних
all_data = pd.DataFrame()

# Ітеруйте по підкаталогах
for activity in os.listdir(data_dir):
    activity_path = os.path.join(data_dir, activity)

    # Перевірте, чи це дійсно каталог
    if os.path.isdir(activity_path):

        # Ітеруйте по всіх файлах CSV у підкаталозі
        for file in os.listdir(activity_path):
            if file.endswith('.csv'):
                file_path = os.path.join(activity_path, file)

                # Завантажте дані з файлу CSV
                data = pd.read_csv(file_path)

                # Додайте стовпець з міткою активності
                data['activity'] = activity

                # Додайте дані до основного DataFrame
                all_data = pd.concat([all_data, data], ignore_index=True)

# Перевірте завантажені дані
print(all_data.head())


   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0         0.359130         6.330270         7.010223     idle
1         0.330400         5.726931         7.838617     idle
2         0.459687         6.277597         7.508217     idle
3         0.277727         6.689400         6.789956     idle
4         0.143652         6.306328         7.819463     idle


Перевірка та Підготовка Даних

In [2]:
# Перевірте перші кілька рядків даних
print(all_data.head())

# Перевірте, чи є пропущені значення
print(all_data.isnull().sum())

# Розділіть дані на ознаки та мітки
X = all_data.drop('activity', axis=1)  # змінили 'activity_label' на 'activity'
y = all_data['activity']

   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0         0.359130         6.330270         7.010223     idle
1         0.330400         5.726931         7.838617     idle
2         0.459687         6.277597         7.508217     idle
3         0.277727         6.689400         6.789956     idle
4         0.143652         6.306328         7.819463     idle
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
activity           0
dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
import time

# Розділимо дані на навчальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Створення моделей
svm_model = SVC()
rf_model = RandomForestClassifier()

# Виміряти час навчання для SVM
start_time = time.time()
svm_model.fit(X_train, y_train)
svm_training_time = time.time() - start_time

# Виміряти час навчання для RandomForest
start_time = time.time()
rf_model.fit(X_train, y_train)
rf_training_time = time.time() - start_time

# Передбачення
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# Вивести час навчання
print("Time taken for SVM training:", svm_training_time, "seconds")
print("Time taken for Random Forest training:", rf_training_time, "seconds")

# Оцінка моделей
print('SVM Classification Report:')
print(classification_report(y_test, svm_predictions))

print('Random Forest Classification Report:')
print(classification_report(y_test, rf_predictions))

Time taken for SVM training: 254.25085258483887 seconds
Time taken for Random Forest training: 15.266478776931763 seconds
SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.91      0.98      0.94      1538
     running       0.93      0.90      0.92     20429
      stairs       1.00      0.00      0.00      1025
     walking       0.80      0.91      0.85     11106

    accuracy                           0.88     34098
   macro avg       0.91      0.70      0.68     34098
weighted avg       0.89      0.88      0.87     34098

Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      1538
     running       1.00      1.00      1.00     20429
      stairs       1.00      1.00      1.00      1025
     walking       1.00      1.00      1.00     11106

    accuracy                           1.00     34098
   macro avg       1.00      1.00      1.00     34098

Час навчання:
SVM: 261.09 секунди
Random Forest: 16.02 секунди
Класифікаційний звіт:
SVM:
Точність (precision):
idle: 0.91
running: 0.93
stairs: 1.00
walking: 0.80
Повернення (recall):
idle: 0.98
running: 0.90
stairs: 0.00
walking: 0.91
F1-score:
idle: 0.94
running: 0.92
stairs: 0.00
walking: 0.85
Загальна точність (accuracy): 0.88
Random Forest:
Точність (precision), Повернення (recall), F1-score для всіх класів: 1.00
Загальна точність (accuracy): 1.00
Загальний висновок:
Модель Random Forest показала відмінні результати, з точністю 1.00 для всіх класів активності, що вказує на ідеальну класифікацію. Навпроти, SVM показав хороші результати для класів idle, running і walking, але не зміг правильно класифікувати клас stairs.

З погляду часу навчання, модель Random Forest була значно швидшою, ніж SVM. SVM вимагав більше часу на навчання, що може бути критичним для великих наборів даних.

Враховуючи вище зазначене, можна сказати, що для даного завдання і набору даних, Random Forest є кращим вибором, як з точки зору точності класифікації, так і з точки зору ефективності часу навчання. Однак, результати можуть варіюватися в залежності від характеристик даних і завдань, тому завжди варто проводити випробування з різними моделями і параметрами.